<a href="https://colab.research.google.com/github/hrumst/ML/blob/master/GAN_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%config IPComplater.greedy = True

In [0]:
from google.colab import drive
drive.mount('./DRIVE', force_remount=True)

In [0]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
tf.enable_eager_execution()

In [0]:
%%time

DATA_SET_PATH_BASE = './DRIVE/My Drive/colab_data/lfw_funneled'
IMG_LEN = 250
IMG_HEIGHT = 250

import os
from PIL import Image
import matplotlib.pyplot as plt

img_paths = []
base_img_paths = os.listdir(DATA_SET_PATH_BASE)
for (i, img_dir) in enumerate(base_img_paths):
    try:
        img_dst = os.path.join(DATA_SET_PATH_BASE, img_dir)
        if img_dst.find('.DS_Store') > -1 or img_dst.find('.txt') > -1:
            continue
        for img in os.listdir(img_dst):
            img_paths.append(os.path.join(img_dst, img))
    except:
        print('err with %' % img_dir)
    if i % 500 == 0:
        print('readed dirs: {} from {}'.format(i, len(base_img_paths)))

In [0]:
%%time

imgs = []
for (i, img_path) in enumerate(img_paths[:200]):
    img = Image.open(img_path)
    imgs.append(np.array(img))
    if i % 100 == 0:
        print('Imported % imgs' % (i))

print(len(imgs))

In [0]:
def plot_digits(samples):
    fig = plt.figure(figsize=(10, 10))
    num = samples.shape[0]
    for j in range(num):
        ax = fig.add_subplot(8, 8, j+1)
        ax.imshow(samples[j, ...].reshape(250, 250, 3), cmap='gray')
        plt.xticks([]), plt.yticks([])
    plt.show()

In [0]:
INPUT_DIM = 100
NUM_EPOCHS = 500
HALF_BATCH_SIZE = 16
BATCH_SIZE = HALF_BATCH_SIZE * 2
LEARNING_RATE = 0.0002

imgs_ds = np.array(imgs).reshape(-1, 250, 250, 3)
train_ds = tf.data.Dataset.from_tensor_slices(imgs_ds)
train_ds = train_ds.shuffle(buffer_size=imgs_ds.shape[0])
train_ds = train_ds.repeat(NUM_EPOCHS)
train_ds = train_ds.batch(HALF_BATCH_SIZE, drop_remainder=True)

optimizer = tf.train.AdamOptimizer(LEARNING_RATE)

In [0]:
tf.reset_default_graph()
generator = tf.keras.Sequential([
    tf.keras.layers.Dense(256*10*10, activation="relu"),
    tf.keras.layers.Reshape((10, 10, 128)),
    tf.keras.layers.UpSampling2D((5, 5)),    
    tf.keras.layers.Conv2D(256, (3, 3), padding="same"),
    tf.keras.layers.BatchNormalization(momentum=0.8),
    tf.keras.layers.ReLU(),    
    tf.keras.layers.UpSampling2D((5, 5)),    
    tf.keras.layers.Conv2D(128, (3, 3), padding="same"),
    tf.keras.layers.BatchNormalization(momentum=0.8),
    tf.keras.layers.ReLU(),    
    tf.keras.layers.Conv2D(3, (3, 3), padding="same", activation='tanh'),
])

discriminator = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(0.2),
    tf.keras.layers.Dropout(0.25),    
    tf.keras.layers.Conv2D(64, kernel_size=3, strides=(2, 2), padding="same"),
    tf.keras.layers.ZeroPadding2D(padding=((0, 1), (0, 1))),
    tf.keras.layers.BatchNormalization(momentum=0.8),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, kernel_size=3, strides=(2, 2), padding="same"),
    tf.keras.layers.BatchNormalization(momentum=0.8),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(256, kernel_size=3, strides=(1, 1), padding="same"),
    tf.keras.layers.BatchNormalization(momentum=0.8),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation=None),
])

In [0]:
for step, true_images in enumerate(train_ds):
    
    # Train Discriminator
    
    noise = np.random.normal(0, 1, (HALF_BATCH_SIZE, INPUT_DIM)).astype(np.float32)
    syntetic_images = generator.predict(noise)
    x_combined = np.concatenate((
        true_images, 
        syntetic_images))
    y_combined = np.concatenate((
        np.ones((HALF_BATCH_SIZE, 1)), 
        np.zeros((HALF_BATCH_SIZE, 1))))
    
    with tf.GradientTape() as tape:
        logits = discriminator(x_combined, training=True)
        d_loss_value = tf.losses.sigmoid_cross_entropy(y_combined, logits)
    grads = tape.gradient(d_loss_value, discriminator.variables)
    optimizer.apply_gradients(zip(grads, discriminator.variables))
    
    # Train Generator
    
    noise = np.random.normal(0, 1, (BATCH_SIZE, INPUT_DIM)).astype(np.float32)
    y_mislabled = np.ones((BATCH_SIZE, 1))
    
    with tf.GradientTape() as tape:
        syntetic = generator(noise, training=True)
        logits = discriminator(syntetic, training=False)
        g_loss_value = tf.losses.sigmoid_cross_entropy(y_mislabled, logits)
    grads = tape.gradient(g_loss_value, generator.variables)
    optimizer.apply_gradients(zip(grads, generator.variables))
    
    # Check intermediate results
    
    if step % 200 == 0:
        print("[Step %2d] D Loss: %.4f; G Loss: %.4f" % (
            step, d_loss_value.numpy(), g_loss_value.numpy()))
        noise = np.random.normal(0, 1, (8, INPUT_DIM)).astype(np.float32)
        syntetic_images = generator.predict(noise)
        plot_digits(syntetic_images)
